**Mount drive to file system**

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

**Unzip images and segmentation data to local drive**

In [ ]:
%cd /content/drive/MyDrive/Machine Learning/


In [ ]:
!mkdir ~/images
!mkdir ~/segm


In [ ]:
!unzip images.zip -d ~/images/
!unzip segm.zip -d ~/segm/

**Install missing dependencies for diffusers**

In [ ]:
!pip install diffusers
!pip install accelerate
!pip install lpips
!pip install deepspeed
!pip install bitsandbytes

**Go to the root project directory**

In [ ]:
%cd "/content/drive/MyDrive/Machine Learning/unet_boundary_xl"

In [ ]:
%load_ext tensorboard

In [ ]:
!rm -rf ./default/logs/
%tensorboard --logdir "./default/logs/seg512"

**Pretrain with simple bottleneck**

--resume_from_checkpoint=latest     resumes from the most recent checkpoint

--checkpoint_to_output              unwraps the model from accelerate and saves it to out_file_name

In [ ]:
!python run.py --output_file_name="output.default.512.params" --checkpoints_total_limit=1  --raw_data_dir="/root/images/images/" --label_data_dir="/root/segm/segm/" --epochs_before_validate=1 --fit_width --train_batch_size=2 --num_train_epochs=3 --learning_rate=1e-3 --output_dir=./default/ --mixed_precision=no --checkpointing_steps=90 --gradient_accumulation_steps=32 --model_bottleneck=default --model_norm=GroupNorm




In [ ]:
!rm -rf ./default_attention/logs/
%tensorboard --logdir "./default_attention/logs/seg512"

**Upgrade Bottleneck**

train just the bottleneck for an epoch

In [ ]:
!python run.py --pretrained_model_path="./default/output.default.512.params" --output_file_name="output.default_attention.512.params" --checkpoints_total_limit=1  --raw_data_dir="/root/images/images/" --label_data_dir="/root/segm/segm/" --epochs_before_validate=1 --fit_width --train_batch_size=1 --num_train_epochs=1 --learning_rate=1e-4 --output_dir=./default_attention/ --mixed_precision=no --checkpointing_steps=90 --gradient_accumulation_steps=64 --model_bottleneck=default --model_norm=GroupNorm --upgrade_bottleneck=attention


**Train Entire Model**

You can probably get away with using a higher training rate here!

In [ ]:
!rm -rf ./attention/logs/
%tensorboard --logdir "./attention/logs/seg512"

In [ ]:
!python run.py --pretrained_model_path="./attention/output.attention.512.params" --output_file_name="output.ft.params" --checkpoints_total_limit=1  --raw_data_dir="/root/images/images/" --label_data_dir="/root/segm/segm/" --epochs_before_validate=1 --fit_width --train_batch_size=1 --num_train_epochs=12 --learning_rate=1e-4 --output_dir=./attention/ --mixed_precision=no --checkpointing_steps=90 --gradient_accumulation_steps=64 --model_bottleneck=attention --model_norm=GroupNorm
